<a href="https://colab.research.google.com/github/mspendyala/aws_bedrock_samples/blob/main/aws_bedrock_example_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os

# Replace with your AWS credentials (use temporary credentials if possible)
os.environ["AWS_ACCESS_KEY_ID"] = ""
os.environ["AWS_SECRET_ACCESS_KEY"] = ""
os.environ["AWS_REGION"] = "us-east-1"

print("AWS credentials configured securely in Colab!")

AWS credentials configured securely in Colab!


In [2]:
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 123.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 8.0 MB/s eta 0:00:00


In [5]:
import boto3
import json
import os
from botocore.exceptions import NoCredentialsError, ClientError

# Load AWS credentials from environment variables (Google Colab or local)
aws_access_key = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
aws_session_token = os.getenv("AWS_SESSION_TOKEN")
aws_region = os.getenv("AWS_REGION", "us-east-1")

# Initialize AWS Clients
s3_client = boto3.client("s3", region_name=aws_region)
bedrock_client = boto3.client("bedrock-runtime", region_name=aws_region)

# S3 Bucket and File
S3_BUCKET = "tesla-support-bot"
S3_FILE_KEY = "tesla_support.json"

def check_s3_file():
    """Check if the knowledge base file exists in S3"""
    try:
        response = s3_client.list_objects_v2(Bucket=S3_BUCKET)
        if "Contents" in response:
            for obj in response["Contents"]:
                if obj["Key"] == S3_FILE_KEY:
                    return True
        print(f"⚠️ File {S3_FILE_KEY} NOT found in S3 bucket `{S3_BUCKET}`.")
        return False
    except ClientError as e:
        print(f"❌ Error checking S3: {e}")
        return False

def fetch_faq_from_s3():
    """Retrieve Tesla support FAQs from S3"""
    try:
        response = s3_client.get_object(Bucket=S3_BUCKET, Key=S3_FILE_KEY)
        faq_data = json.loads(response["Body"].read().decode("utf-8"))
        return faq_data
    except NoCredentialsError:
        print("❌ AWS credentials not found!")
        return []
    except ClientError as e:
        print(f"❌ Error fetching knowledge base: {e}")
        return []

def retrieve_faq_answer(user_question, faq_data):
    """Retrieve the best-matching FAQ from the knowledge base"""
    for entry in faq_data:
        if user_question.lower() in entry["question"].lower():
            return entry["answer"]
    return "No relevant answer found."

def query_bedrock(prompt):
    """Query Amazon Bedrock with the correct prompt format for Claude"""
    formatted_prompt = f'\n\nHuman: {prompt}\n\nAssistant:'  # 👈 Add 'Human:' prefix

    try:
        response = bedrock_client.invoke_model(
            modelId="anthropic.claude-instant-v1",  # Change model if needed
            body=json.dumps({
                "prompt": formatted_prompt,
                "max_tokens_to_sample": 250  # Adjust token limit
            })
        )
        result = json.loads(response["body"].read().decode("utf-8"))
        return result.get("completion", "No response from model")
    except ClientError as e:
        print(f"❌ Error calling Bedrock: {e}")
        return "Error in Bedrock call."

def main():
    """Main workflow for Tesla Support Chatbot"""
    user_query = "How do I enable Autopilot in my Tesla?"

    # Step 1: Check if the S3 knowledge base exists
    if not check_s3_file():
        return

    # Step 2: Fetch Tesla support FAQs from S3
    faq_data = fetch_faq_from_s3()

    # Step 3: Retrieve relevant FAQ answer
    retrieved_answer = retrieve_faq_answer(user_query, faq_data)

    # Step 4: Create prompts for Bedrock
    vanilla_prompt = f"How do I enable Autopilot in my Tesla?"
    rag_prompt = f"Using the following product support information:\n{retrieved_answer}\n\nAnswer this: {user_query}"

    # Step 5: Query Bedrock
    response_vanilla = query_bedrock(vanilla_prompt)
    response_rag = query_bedrock(rag_prompt)

    # Step 6: Display results
    print("\n🔹 **Vanilla Bedrock Response:**")
    print(response_vanilla)
    print("\n🔹 **RAG-Enhanced Response (Using S3):**")
    print(response_rag)

if __name__ == "__main__":
    main()



🔹 **Vanilla Bedrock Response:**
 Here are the basic steps to enable Autopilot in a Tesla, but keep in mind that actual functionality and requirements may vary depending on the specific vehicle model and version of the Autopilot software:

1. Make sure your vehicle is equipped with Autopilot hardware. This includes cameras mounted around the car, radar sensors, and other components. Newer Tesla models come standard with Autopilot, but it may need to be purchased as an add-on for some older models.

2. Ensure your Tesla account shows that you have purchased or leased a vehicle with the Autopilot feature enabled. You'll generally need to pay an additional fee to get access to Autopilot. 

3. Update your Tesla to the latest version of the Autopilot software. Over-the-air software updates are required for new Autopilot features.

4. Familiarize yourself with how to engage Autopilot by watching the onboard tutorials or Tesla's website guides. You'll typically use the Autopilot engagement bu